# Все блоки за день по одному каналу
Пример выгрузки всех блоков за один день по одному каналу 

## Описание задачи и условий расчета
- Период: 05.06.2023
- ЦА: Все 14-44
- Место просмотра: Все места (Дом+Дача)
- Каналы: ТНТ
- Блоки
- Статистики: Rtg%

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

## Справочники
Получим идентификаторы, которые будут использоваться для формирования условий расчета

In [ ]:
# Найдем id телекомпании ТНТ (СЕТЕВОЕ ВЕЩАНИЕ)
cats.get_tv_company(name=['ТНТ (СЕТЕВОЕ ВЕЩАНИЕ)'])

# id = 1870

## Формирование задания
Зададим условия расчета

In [ ]:
# Задаем период
# Период указывается в виде списка ('Начало', 'Конец') в формате 'YYYY-MM-DD'. Можно указать несколько периодов
date_filter = [('2023-06-05', '2023-06-05')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем ЦА - Все 14-44
basedemo_filter = 'age >= 14 and age <= 44'

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter=None

# Задаем каналы - ТНТ
company_filter = 'tvCompanyId = 1870'

# Фильтр программ
program_filter = None

# Фильтр блоков
break_filter = None

# Фильтр роликов
ad_filter = None

# Указываем список срезов
slices = ['breaksSpotId', #Блок ID выхода, обязательный атрибут! 
          'researchDate', #Дата, обязательный атрибут!
          'tvCompanyName', #Телекомпания
          'breaksStartTime', #Блок время начала
          'breaksFinishTime', #Блок время окончания
          'breaksName' #Блок
         ]
# Указываем список статистик для расчета
statistics = ['RtgPer']

# Задаем условия сортировки: дата (от старых к новым), время начала (по возр.)
sortings = {'researchDate':'ASC', 'breaksStartTime':'ASC'}

# Задаем опции расчета
options = {
    "kitId": 1 #TV Index Russia all
}

## Расчет задания

In [ ]:
# Формируем задание для API TV Index в формате JSON
task_json = mtask.build_simple_task(date_filter=date_filter, weekday_filter=weekday_filter, 
                                        daytype_filter=daytype_filter, company_filter=company_filter, 
                                        location_filter=location_filter, basedemo_filter=basedemo_filter, 
                                        targetdemo_filter=targetdemo_filter,program_filter=program_filter, 
                                        break_filter=break_filter, ad_filter=ad_filter, 
                                        slices=slices, statistics=statistics, sortings=sortings, options=options)

# Отправляем задание на расчет и ждем выполнения
task_simple = mtask.wait_task(mtask.send_simple_task(task_json))

# Получаем результат
df = mtask.result2table(mtask.get_result(task_simple))

In [ ]:
# Приводим порядок столбцов в соответствие с условиями расчета
df = df[slices+statistics]
df

## Сохраняем в Excel
По умолчанию файл сохраняется в папку `excel`

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('01_all_breaks_one_day_one_channel')) as writer:
    df.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)